<a href="https://colab.research.google.com/github/fsndzomga/Deep-Learning-With-Python/blob/main/Transformers_c1_c2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install transformers

In [ ]:
# Initialize the zero-shot classification pipeline
classifier = pipeline("zero-shot-classification")

# Define the text to classify
text_to_classify = "I love programming."

# Define the categories (labels) you're interested in
categories = ['entertainment', 'technology', 'sports', 'politics']

# Perform the classification
result = classifier(text_to_classify, candidate_labels=categories)

# Print the result
print(result)



In [4]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis")


classifier("I hate my current situation.")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


[{'label': 'NEGATIVE', 'score': 0.9994239807128906}]

In [5]:
classifier(
    ["I've been waiting for a HuggingFace course my whole life.", "I hate this so much!"]
)

[{'label': 'POSITIVE', 'score': 0.9598048329353333},
 {'label': 'NEGATIVE', 'score': 0.9994558691978455}]

In [6]:
from transformers import pipeline

generator = pipeline("text-generation")
generator("In this course, we will teach you how to")

No model was supplied, defaulted to gpt2 and revision 6c0e608 (https://huggingface.co/gpt2).
Using a pipeline without specifying a model name and revision in production is not recommended.


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'In this course, we will teach you how to manage a lot of the data you need, how to use RESTful Data Storage for managing your data and how to optimize your data plan.\n\nHow do I get started with SQLite and other'}]

In [7]:
from transformers import pipeline

generator = pipeline("text-generation", model="distilgpt2")
generator(
    "In this course, we will teach you how to",
    max_length=30,
    num_return_sequences=2,
)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'In this course, we will teach you how to make a good difference. In this course we offer you the necessary knowledge to overcome obstacles and overcome obstacles'},
 {'generated_text': 'In this course, we will teach you how to handle the problem in the real world.\nThis course is an introduction to the practical side of the'}]

In [ ]:
from transformers import AutoTokenizer

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)

raw_inputs = [
    "I've been waiting for a HuggingFace course my whole life.",
    "I hate this so much!",
]

inputs = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors="pt")

print(inputs)

In [ ]:
from transformers import AutoModel

model = AutoModel.from_pretrained(checkpoint)

In [ ]:
outputs = model(**inputs)
print(outputs.last_hidden_state.shape)

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

outputs = model(**inputs)


In [ ]:
print(outputs.logits.shape)

In [ ]:
print(outputs.logits)

In [ ]:
import torch

predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)

print(predictions)

In [ ]:
model.config.id2label

In [ ]:
model.config

In [ ]:
from transformers import BertConfig, BertModel

#Building the config
config = BertConfig()

model = BertModel(config)

print(config)

In [ ]:
from transformers import BertModel

model = BertModel.from_pretrained("bert-base-cased")


In [ ]:
sequences = ["Hello!", "Cool.", "Nice!"]

encoded_sequences = [
    [101, 7592, 999, 102],
    [101, 4658, 1012, 102],
    [101, 3835, 999, 102],
]

import torch

model_inputs = torch.tensor(encoded_sequences)

output = model(model_inputs)

In [ ]:
print(output)

In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-cased")

tokenizer("Learn about transformers not langchain")

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)

model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

sequence = ["I want to master NLP and use it in my web projects.",
            "I love programming",
            ]

all_ids = []

for seq in sequence:
  tokens = tokenizer.tokenize(seq)

  ids = tokenizer.convert_tokens_to_ids(tokens)

  all_ids.append(ids)

print(all_ids)

batched_ids = [[1045, 2215, 2000, 3040, 17953, 2361, 1998, 2224, 2009, 1999, 2026, 4773, 3934, 1012],
               [1045, 2293, 4730, tokenizer.pad_token_id, tokenizer.pad_token_id, tokenizer.pad_token_id,tokenizer.pad_token_id,tokenizer.pad_token_id,tokenizer.pad_token_id,tokenizer.pad_token_id,tokenizer.pad_token_id,tokenizer.pad_token_id,tokenizer.pad_token_id,tokenizer.pad_token_id]]

attention_mask = [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
                  [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]


outputs = model(torch.tensor(batched_ids), attention_mask=torch.tensor(attention_mask))

print(outputs.logits)


In [ ]:
from transformers import AutoTokenizer

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)

sequence = "I am learning programming and want to excel at it."

model_inputs = tokenizer(sequence)

print(model_inputs)

Sentiment Analysis using the transformers library

In [ ]:
import pandas as pd

df = pd.read_csv('amazon_cells_labelled.txt', delimiter='\t', header=None, names=['Review', 'Sentiment'])

In [8]:
from transformers import pipeline
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import pandas as pd

df = pd.read_csv('amazon_cells_labelled.txt', delimiter='\t', header=None, names=['Review', 'Sentiment'])

classifier = pipeline("sentiment-analysis")

predicted_sentiments = []

for index, row in df.iterrows():
  result = classifier(row['Review'])
  if result[0]['label'] == 'POSITIVE':
    sentiment = 1
  else:
    sentiment = 0
  predicted_sentiments.append(sentiment)

df['predicted_sentiment'] = predicted_sentiments

# Evaluate the performance using accuracy, confusion matrix, and classification report
accuracy = accuracy_score(df['Sentiment'], df['predicted_sentiment'])
conf_matrix = confusion_matrix(df['Sentiment'], df['predicted_sentiment'])
class_report = classification_report(df['Sentiment'], df['predicted_sentiment'])

accuracy, conf_matrix, class_report





No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


(0.925,
 array([[468,  32],
        [ 43, 457]]),
 '              precision    recall  f1-score   support\n\n           0       0.92      0.94      0.93       500\n           1       0.93      0.91      0.92       500\n\n    accuracy                           0.93      1000\n   macro avg       0.93      0.93      0.92      1000\nweighted avg       0.93      0.93      0.92      1000\n')